In [ ]:
# Importing necessary packages
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/preprocessed_translated_samples.csv')


In [ ]:
# Data preprocessing
import nltk
import re
import string
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer

In [ ]:
# Downloading stopwords and initializing stemmer
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))
stemmer = ISRIStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Preprocessing function for Arabic text
def preprocessor(text):
    # Removing non-alphabetic characters and digits
    text = re.sub(r'[^ء-ي\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Removing stopwords and applying stemming
    processed_text = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(processed_text)

In [ ]:
df.isnull().sum()

Translated_Tweet    0
Suicide             0
dtype: int64

In [ ]:
# Dropping rows with empty 'Translated_Tweet' or 'Suicide'
df.dropna(subset=['Translated_Tweet', 'Suicide'], inplace=True)

In [ ]:
df.isnull().sum()

Translated_Tweet    0
Suicide             0
dtype: int64

In [ ]:
# Applying preprocessing to the 'Translated_Tweet' column
df['Translated_Tweet'] = df['Translated_Tweet'].apply(preprocessor)

In [ ]:
# Encoding labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Suicide'] = encoder.fit_transform(df['Suicide'])

In [ ]:
# Addressing class imbalance using SMOTE (Synthetic Minority Over-sampling Technique)
from imblearn.over_sampling import SMOTE

# Splitting data into features and labels
X = df['Translated_Tweet']
y = df['Suicide']

In [ ]:
# Splitting data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Translated_Tweet'], df['Suicide'], test_size=0.2, random_state=1)


In [ ]:
# Vectorizing text using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Apply SMOTE to training data
smote = SMOTE(random_state=1)
X_train_tfidf_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

In [ ]:
# Model Building
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

rf_classifier = RandomForestClassifier(class_weight='balanced', random_state=1)
rf_classifier.fit(X_train_tfidf_smote, y_train_smote)
y_pred = rf_classifier.predict(X_test_tfidf)

In [ ]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8263305322128851
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.94      0.87       225
           1       0.86      0.64      0.73       132

    accuracy                           0.83       357
   macro avg       0.84      0.79      0.80       357
weighted avg       0.83      0.83      0.82       357

Confusion Matrix:
[[211  14]
 [ 48  84]]


In [ ]:
# Function to evaluate a single input text
def evaluate_input(text):
    # Preprocess the input text
    processed_text = preprocessor(text)
    # Vectorize the input text
    vectorized_text = tfidf_vectorizer.transform([processed_text])
    # Predict the label
    prediction = rf_classifier.predict(vectorized_text)
    # Decode the label
    decoded_prediction = encoder.inverse_transform(prediction)
    return decoded_prediction[0]

In [ ]:
# Example usage of the evaluation function with multiple inputs
texts = [
    "أشعر بالإحباط الشديد ولا أرى أي أمل في المستقبل",
    "أنا سعيد جدا اليوم وأحب حياتي",
    "لا أستطيع تحمل هذا الألم أكثر"
]

for text in texts:
    result = evaluate_input(text)
    print(f"Input text: {text}\nPrediction: {result}\n")


Input text: أشعر بالإحباط الشديد ولا أرى أي أمل في المستقبل
Prediction: Potential Suicide post 

Input text: أنا سعيد جدا اليوم وأحب حياتي
Prediction: Not Suicide post

Input text: لا أستطيع تحمل هذا الألم أكثر
Prediction: Potential Suicide post 



In [ ]:
# Example usage of the evaluation function with multiple inputs
texts = [
    "أشعر بالإحباط الشديد ولا أرى أي أمل في المستقبل",  # Potential Suicide post
    "أنا سعيد جدا اليوم وأحب حياتي",  # Not Suicide post
    "لا أستطيع تحمل هذا الألم أكثر",  # Potential Suicide post
    "اليوم كان يوما جميلا وأستمتع به",  # Not Suicide post
    "أفكر في إنهاء حياتي لأنني لا أستطيع المواصلة",  # Potential Suicide post
    "استمتعت بلقاء أصدقائي والحديث معهم",  # Not Suicide post
    "أنا مكتئب ولا أستطيع التحمل",  # Potential Suicide post
    "أحب عائلتي وأصدقائي وهم يدعمونني دائما",  # Not Suicide post
    "هذا الألم لا يطاق وأريد التخلص منه",  # Potential Suicide post
    "لقد أنجزت الكثير من الأشياء الجيدة هذا الأسبوع",  # Not Suicide post
    "لا أستطيع العثور على أي سبب للاستمرار",  # Potential Suicide post
    "أخطط لعطلة رائعة مع عائلتي",  # Not Suicide post
    "كل شيء يبدو مظلما ولا أستطيع رؤية النور في نهاية النفق",  # Potential Suicide post
    "أنا ممتن لكل الأشياء الجيدة في حياتي",  # Not Suicide post
]

for text in texts:
    result = evaluate_input(text)
    print(f"Input text: {text}\nPrediction: {result}\n")

Input text: أشعر بالإحباط الشديد ولا أرى أي أمل في المستقبل
Prediction: Potential Suicide post 

Input text: أنا سعيد جدا اليوم وأحب حياتي
Prediction: Not Suicide post

Input text: لا أستطيع تحمل هذا الألم أكثر
Prediction: Potential Suicide post 

Input text: اليوم كان يوما جميلا وأستمتع به
Prediction: Not Suicide post

Input text: أفكر في إنهاء حياتي لأنني لا أستطيع المواصلة
Prediction: Potential Suicide post 

Input text: استمتعت بلقاء أصدقائي والحديث معهم
Prediction: Not Suicide post

Input text: أنا مكتئب ولا أستطيع التحمل
Prediction: Potential Suicide post 

Input text: أحب عائلتي وأصدقائي وهم يدعمونني دائما
Prediction: Not Suicide post

Input text: هذا الألم لا يطاق وأريد التخلص منه
Prediction: Potential Suicide post 

Input text: لقد أنجزت الكثير من الأشياء الجيدة هذا الأسبوع
Prediction: Not Suicide post

Input text: لا أستطيع العثور على أي سبب للاستمرار
Prediction: Not Suicide post

Input text: أخطط لعطلة رائعة مع عائلتي
Prediction: Not Suicide post

Input text: كل شيء يبدو مظل